In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000
adapt_c = False


size weirdness rn...

In [5]:
a = 0.25
num_params=9

calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/10) )
c1,c2 = calc_c(nobs)

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=500,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.63, 5th: 4.01, 10th: 4.95, 30th: 7.40, 50th: 9.50, 70th: 12.18, 90th: 16.54, 95th: 19.05, 99th: 24.02
bonus term 0.0018592550410762333 // variance stats 0.3390440700632484 //part1 1.9952623149688797 //part2 0.0009318348906445578
-2.0437861351072795 0.16160633423780701 1.9219833802507957
-2.047504645189432 0.16160633423780701 1.918264870168643
---
Quantiles: 1th: 2.57, 5th: 3.98, 10th: 4.91, 30th: 7.38, 50th: 9.46, 70th: 11.94, 90th: 16.27, 95th: 18.87, 99th: 23.99
bonus term 0.0018927763115621876 // variance stats 0.33303396297017096 //part1 1.9952623149688797 //part2 0.0009486353234670848
-2.0937233380317863 -0.47997899371319275 2.0750657518840514
-2.097508890654911 -0.47997899371319275 2.0712801992609267
---
Quantiles: 1th: 2.75, 5th: 4.13, 10th: 4.94, 30th: 7.30, 50th: 9.43, 70th: 11.96, 90th: 16.17, 95th: 18.45, 99th: 23.93
bonus term 0.0018231053247963857 // variance stats 0.34577312642456565 //part1 1.9952623149688797 //part2 0.0009137171143458503
-1.97698966904

# power 0.... need to see...

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:

data_tuned_c = .8
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.62, 5th: 4.07, 10th: 5.00, 30th: 7.64, 50th: 10.03, 70th: 12.76, 90th: 17.44, 95th: 20.29, 99th: 26.15
bonus term 0.06937694637593697 // variance stats 0.39142320931891217 //part1 3.017088168272582 //part2 0.022994669862650512
-1.8099007983414566 2.5239927280358585 2.2024177718759375
-1.9486546910933307 2.5239927280358585 2.063663879124064
---
Quantiles: 1th: 2.51, 5th: 3.81, 10th: 4.75, 30th: 7.17, 50th: 9.38, 70th: 12.24, 90th: 17.49, 95th: 20.17, 99th: 26.97
bonus term 0.06516297175055753 // variance stats 0.40807295302907554 //part1 3.017088168272582 //part2 0.021597967350044746
-1.797391010851089 3.5146853750381304 2.1257644390632864
-1.9277169543522041 3.5146853750381304 1.995438495562171
---
Quantiles: 1th: 2.76, 5th: 4.07, 10th: 5.00, 30th: 7.54, 50th: 9.82, 70th: 12.69, 90th: 17.78, 95th: 21.30, 99th: 28.71
bonus term 0.06718743441232465 // variance stats 0.40681363069187815 //part1 3.017088168272582 //part2 0.022268966190269628
-1.8476077552218821 1.53085845

In [8]:

data_tuned_c = .95
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/3) )

nobs = 250
c1,c2 = calc_c(nobs)

a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.38, 5th: 3.79, 10th: 4.68, 30th: 7.15, 50th: 9.47, 70th: 12.56, 90th: 18.19, 95th: 21.60, 99th: 28.67
bonus term 0.26784738224723165 // variance stats 0.3076151198864181 //part1 6.299605249474365 //part2 0.0425181216346184
-1.6888914130550325 1.4137559811387261 2.170965857634719
-2.224586177549496 1.4137559811387261 1.6352710931402559
---
Quantiles: 1th: 2.60, 5th: 3.96, 10th: 5.04, 30th: 7.72, 50th: 10.12, 70th: 13.16, 90th: 19.08, 95th: 22.53, 99th: 29.85
bonus term 0.2048992821295409 // variance stats 0.4238008943890807 //part1 6.299605249474365 //part2 0.032525733599996214
-1.662005264055575 0.6548028553500593 1.9960423948801156
-2.071803828314657 0.6548028553500593 1.586243830621034
---
Quantiles: 1th: 2.71, 5th: 4.02, 10th: 4.89, 30th: 7.35, 50th: 9.53, 70th: 12.33, 90th: 17.34, 95th: 20.08, 99th: 26.87
bonus term 0.17071483557590814 // variance stats 0.4559382235920726 //part1 6.299605249474365 //part2 0.027099290957977484
-1.6833177594378659 1.0440901249240828

In [9]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/250,nobs**(1/3) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.35, 5th: 3.62, 10th: 4.57, 30th: 6.90, 50th: 9.19, 70th: 11.98, 90th: 17.10, 95th: 19.85, 99th: 26.52
bonus term 0.0911332722183235 // variance stats 0.2946878702251134 //part1 6.299605249474365 //part2 0.014466505218867104
-1.864183055612935 1.6410777497829545 2.06867533204274
-2.0464496000495824 1.6410777497829545 1.8864087876060927
---
Quantiles: 1th: 2.80, 5th: 4.51, 10th: 5.60, 30th: 8.71, 50th: 11.42, 70th: 14.70, 90th: 20.79, 95th: 23.94, 99th: 31.29
bonus term 0.10460194476207847 // variance stats 0.3114536302019385 //part1 6.299605249474365 //part2 0.016604523715324285
-1.8792543418730212 1.8079861450344659 1.9176744326643067
-2.088458231397178 1.8079861450344659 1.7084705431401497
---
Quantiles: 1th: 2.93, 5th: 4.42, 10th: 5.42, 30th: 8.22, 50th: 10.66, 70th: 13.65, 90th: 19.36, 95th: 22.50, 99th: 30.35
bonus term 0.09476804679409594 // variance stats 0.32075401205099435 //part1 6.299605249474365 //part2 0.015043489717391945
-1.8292000059391909 1.47730289273

In [10]:
data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/10) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.86, 5th: 4.17, 10th: 5.17, 30th: 7.89, 50th: 10.24, 70th: 13.36, 90th: 18.95, 95th: 22.14, 99th: 28.67
bonus term 0.06625442547798124 // variance stats 0.30229009507814336 //part1 1.736976732219687 //part2 0.03814353079635934
-1.7220826484126215 2.547754835602993 1.8939037568290542
-1.8545914993685841 2.547754835602993 1.7613949058730916
---
Quantiles: 1th: 2.42, 5th: 3.74, 10th: 4.56, 30th: 6.94, 50th: 9.13, 70th: 11.84, 90th: 16.72, 95th: 19.49, 99th: 26.16
bonus term 0.058426160915717784 // variance stats 0.30379743905573825 //part1 1.736976732219687 //part2 0.03363669750547253
-1.8513221645366622 0.5461129299520454 2.0018925586234757
-1.9681744863680977 0.5461129299520454 1.8850402367920398
---
Quantiles: 1th: 2.89, 5th: 4.47, 10th: 5.64, 30th: 8.66, 50th: 11.39, 70th: 14.53, 90th: 20.64, 95th: 24.12, 99th: 32.28
bonus term 0.06793527978173379 // variance stats 0.3206769613077406 //part1 1.736976732219687 //part2 0.03911122038745972
-1.835262187963596 1.3272877211

In [11]:

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/50,nobs**(1/10) )

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(c1,c2)
print(mc_out)

Quantiles: 1th: 2.57, 5th: 3.89, 10th: 4.87, 30th: 7.55, 50th: 10.06, 70th: 12.96, 90th: 18.15, 95th: 20.92, 99th: 26.84
bonus term 0.11608158594447843 // variance stats 0.3205917562822967 //part1 1.736976732219687 //part2 0.06682967237916738
-1.7420309003491825 0.8068435887941083 2.011883598245435
-1.9741940722381393 0.8068435887941083 1.779720426356478
---
Quantiles: 1th: 2.35, 5th: 3.79, 10th: 4.79, 30th: 7.33, 50th: 9.58, 70th: 12.45, 90th: 17.77, 95th: 20.58, 99th: 27.39
bonus term 0.11414149515227975 // variance stats 0.3196077931997609 //part1 1.736976732219687 //part2 0.06571273698434524
-1.7033609512749017 0.6045795085321235 2.0434943755178003
-1.931643941579461 0.6045795085321235 1.815211385213241
---
Quantiles: 1th: 2.36, 5th: 3.79, 10th: 4.59, 30th: 6.95, 50th: 9.02, 70th: 11.53, 90th: 15.97, 95th: 18.39, 99th: 24.47
bonus term 0.07392955769638376 // variance stats 0.45432712194431574 //part1 1.736976732219687 //part2 0.04256220381369703
-1.792814480220537 0.866284301195177

# a  = .25, k= 9

In [12]:
a = 0.25
num_params=9



In [13]:
nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.93, 5th: 4.48, 10th: 5.58, 30th: 8.45, 50th: 10.99, 70th: 14.24, 90th: 20.30, 95th: 23.88, 99th: 31.70
bonus term 0.08865074202948581 // variance stats 0.47740031055477505 //part1 1.736976732219687 //part2 0.051037380285571705
-1.8889800279315825 -0.8350931647890311 1.944368169348506
-2.066281511990554 -0.8350931647890311 1.7670666852895345
---
Quantiles: 1th: 2.66, 5th: 4.27, 10th: 5.32, 30th: 8.14, 50th: 10.68, 70th: 13.83, 90th: 19.98, 95th: 23.21, 99th: 30.93
bonus term 0.10768343340674627 // variance stats 0.38241599835470425 //part1 1.736976732219687 //part2 0.06199474720029055
-1.6919977800005315 0.5075513940039231 1.961007426335696
-1.907364646814024 0.5075513940039231 1.7456405595222035
---
Quantiles: 1th: 2.45, 5th: 3.86, 10th: 4.75, 30th: 7.37, 50th: 9.74, 70th: 12.69, 90th: 18.58, 95th: 22.00, 99th: 29.04
bonus term 0.11540581864428441 // variance stats 0.33029523905952635 //part1 1.736976732219687 //part2 0.06644062439270963
-1.8288154601663986 0.07741390

In [14]:

nobs = 500
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,print_stuff=True)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.60, 5th: 4.21, 10th: 5.30, 30th: 7.86, 50th: 10.18, 70th: 12.96, 90th: 18.16, 95th: 20.90, 99th: 26.35
bonus term 0.006111999963050933 // variance stats 0.27153790672095474 //part1 1.8616455666360694 //part2 0.003283116868532129
-1.9810885647349514 1.0252925013613878 1.9503403732482891
-1.9933125646610534 1.0252925013613878 1.9381163733221871
---
Quantiles: 1th: 2.65, 5th: 4.02, 10th: 4.86, 30th: 7.33, 50th: 9.54, 70th: 12.04, 90th: 16.51, 95th: 18.98, 99th: 25.02
bonus term 0.006146944579220575 // variance stats 0.26998916275262047 //part1 1.8616455666360694 //part2 0.003301887689786137
-1.8472199660781685 0.11478600683282877 2.019145076402673
-1.8595138552366097 0.11478600683282877 2.0068511872442323
---
Quantiles: 1th: 2.82, 5th: 4.26, 10th: 5.26, 30th: 7.96, 50th: 10.25, 70th: 12.92, 90th: 17.98, 95th: 21.00, 99th: 26.98
bonus term 0.0038330531077286232 // variance stats 0.4335129154691828 //part1 1.8616455666360694 //part2 0.002058959651839002
-1.8055950903576532

In [15]:
nobs = 100
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=True, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

Quantiles: 1th: 2.13, 5th: 3.47, 10th: 4.29, 30th: 6.54, 50th: 8.78, 70th: 11.39, 90th: 16.69, 95th: 20.24, 99th: 28.37
bonus term 0.12238133609645936 // variance stats 0.39900632178909207 //part1 1.5848931924611136 //part2 0.07721740283736002
-1.848915280190039 0.410831270637307 1.8873109982449796
-2.093677952382958 0.410831270637307 1.6425483260520608
---
Quantiles: 1th: 2.52, 5th: 3.99, 10th: 5.03, 30th: 7.88, 50th: 10.83, 70th: 14.93, 90th: 24.07, 95th: 30.25, 99th: 46.28
bonus term 0.139277041151039 // variance stats 0.4997227976060082 //part1 1.5848931924611136 //part2 0.08787787203171817
-1.8542795376412862 -0.20615584002941237 2.004447888840554
-2.1328336199433644 -0.20615584002941237 1.7258938065384763
---
Quantiles: 1th: 2.63, 5th: 3.99, 10th: 4.95, 30th: 8.00, 50th: 10.88, 70th: 14.67, 90th: 22.15, 95th: 26.72, 99th: 38.33
bonus term 0.15590885776744443 // variance stats 0.4061235658073643 //part1 1.5848931924611136 //part2 0.09837183887788688
-1.664948076700715 -0.166328282

# a  = .25, k= 4, something not right?

In [16]:
a = 0.25
num_params=4



In [17]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.98 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.02 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.96 , 0.006, 0.034]), array([0.96 , 0.006, 0.034]), array([0.978, 0.002, 0.02 ]), array([0.976, 0.012, 0.012]), array([0.98, 0.01, 0.01]), array([0.986, 0.008, 0.006]), -1.9701371208927592, 5.316941569895832, 5.603483610033619)


# a  = .25, K = 19

In [18]:
a = 0.25
num_params=19


In [19]:

nobs = 250
c1,c2 = calc_c(nobs)


setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.72 & 0.80 & 0.74 & 0.97 & 0.98 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.01 & 0.01   \\
Model 2 & 0.28 & 0.20 & 0.26 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.716, 0.   , 0.284]), array([0.802, 0.   , 0.198]), array([0.74, 0.  , 0.26]), array([0.972, 0.016, 0.012]), array([0.978, 0.012, 0.01 ]), array([0.974, 0.014, 0.012]), -9.307268447330898, 6.3888582733323505, 6.715882835185446)


# evidence of power

In [20]:

nobs = 500
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.98 & 0.97 & 0.64 & 0.72 & 0.64   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.36 & 0.28 & 0.36   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.974, 0.02 , 0.006]), array([0.976, 0.02 , 0.004]), array([0.972, 0.02 , 0.008]), array([0.638, 0.362, 0.   ]), array([0.72, 0.28, 0.  ]), array([0.638, 0.362, 0.   ]), 1.6137868147037553, 5.653047798560255, 6.24755052434197)


In [21]:

nobs = 100
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 1.00 & 0.97 & 0.68 & 0.76 & 0.69   \\
Model 1 & 0.02 & 0.00 & 0.02 & 0.32 & 0.23 & 0.31   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.966, 0.022, 0.012]), array([1., 0., 0.]), array([0.972, 0.02 , 0.008]), array([0.682, 0.316, 0.002]), array([0.764, 0.234, 0.002]), array([0.692, 0.306, 0.002]), 0.46767916058932607, 5.753658908544015, 5.799287734681068)


In [22]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.58 & 0.58 & 0.61 & 0.44 & 0.48 & 0.54   \\
Model 1 & 0.42 & 0.42 & 0.39 & 0.56 & 0.52 & 0.46   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.578, 0.422, 0.   ]), array([0.58, 0.42, 0.  ]), array([0.612, 0.388, 0.   ]), array([0.438, 0.562, 0.   ]), array([0.478, 0.522, 0.   ]), array([0.536, 0.464, 0.   ]), 9.156707403059656, 4.851951960817179, 4.912298256557861)


In [23]:

nobs = 250
c1,c2 = calc_c(nobs)


a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.98 & 0.97 & 0.65 & 0.73 & 0.65   \\
Model 1 & 0.02 & 0.02 & 0.02 & 0.35 & 0.27 & 0.35   \\
Model 2 & 0.00 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.974, 0.022, 0.004]), array([0.978, 0.022, 0.   ]), array([0.974, 0.02 , 0.006]), array([0.648, 0.352, 0.   ]), array([0.73, 0.27, 0.  ]), array([0.654, 0.346, 0.   ]), 1.7759179353528909, 5.575769877491148, 6.168017093182235)


# evidence of power 2

In [24]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.55 & 0.02 & 0.58 & 0.69 & 0.60   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.45 & 0.98 & 0.42 & 0.31 & 0.40   \\
\hline
\end{tabular}
(array([0.008, 0.   , 0.992]), array([0.546, 0.   , 0.454]), array([0.02, 0.  , 0.98]), array([0.58, 0.  , 0.42]), array([0.694, 0.   , 0.306]), array([0.596, 0.   , 0.404]), -20.698494232759206, 5.625520809075673, 6.208665207428883)


In [25]:
nobs = 500
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.38 & 0.02 & 0.66 & 0.74 & 0.69   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.62 & 0.98 & 0.34 & 0.26 & 0.31   \\
\hline
\end{tabular}
(array([0.014, 0.   , 0.986]), array([0.378, 0.   , 0.622]), array([0.016, 0.   , 0.984]), array([0.658, 0.   , 0.342]), array([0.742, 0.   , 0.258]), array([0.686, 0.   , 0.314]), -20.01296523331106, 5.600464671093142, 6.275069335800621)


In [ ]:
nobs = 100
c1,c2 = calc_c(nobs)



a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250
c1,c2 = calc_c(nobs)


a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

In [ ]:
nobs = 250
c1,c2 = calc_c(nobs)



a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c,
                                     print_stuff=False, data_tuned_c=data_tuned_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)